In [11]:
# Instalamos Los paquetes necesarios.
!sudo apt install tesseract-ocr
!pip install pytesseract
!apt-get install poppler-utils
!pip install pdf2image
!apt-get install -y libpoppler-cpp-dev
!pip install --upgrade pdf2image
!pip install python-Levenshtein
!pip install pypdf
! wget https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/spa_old.traineddata #https://stackoverflow.com/questions/57968216/google-colab-how-do-i-install-traineddata-file-for-pytesseract
! sudo mv "/content/spa_old.traineddata" "/usr/share/tesseract-ocr/4.00/tessdata"
! wget https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/spa.traineddata #https://stackoverflow.com/questions/57968216/google-colab-how-do-i-install-traineddata-file-for-pytesseract
! sudo mv "/content/spa.traineddata" "/usr/share/tesseract-ocr/4.00/tessdata"

# Get the scripts with the functions from github https://github.com/jisalascaceres/Apoyo-Filologia
! wget https://raw.githubusercontent.com/jisalascaceres/Apoyo-Filologia/main/Contador_Palabras/Word_Count.py
! wget https://raw.githubusercontent.com/jisalascaceres/Apoyo-Filologia/main/Contador_Palabras/Utils.py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libpoppler-cpp-dev is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
--2024-12-15 20:20:14--  https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/spa_old.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.1

In [12]:
import pandas as pd
import numpy as np
import sys
import random
import pypdf
from tqdm import tqdm
import os
import time
import Levenshtein
from nltk.tokenize import word_tokenize
import pytesseract
from pdf2image import convert_from_path
from pypdf import PdfReader
import nltk
nltk.download('punkt_tab')

from Utils import *
from Word_count import *

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [15]:
name_excel = 'antillanismos.xlsx'
name_pdf = '1637-Moreno.pdf'
path_output = 'Output.xlsx'

path_csv = '/content/' + name_excel
path_pdf = '/content/' + name_pdf

start_page = 1
end_page = -1
max_distance = 2
return_coincidences = True
formato = '.xlsx'
number_of_returns = 2
folder = False



Perform_word_count(path_pdf, path_csv, start_page, end_page, max_distance, return_coincidences, path_output, formato,
                       number_of_returns, folder,'spa')


Iniciando análisis de texto...
PDF:  /content/1637-Moreno.pdf
La página final debe ser menor que el número de páginas del pdf, que es 73, se establece el parámetro a este último.
Número de páginas del PDF:  73
Leyendo desde la página 1 hasta la página 73
Digitalizando el texto...
Esto puede tardar unos minutos, por favor, espere.


100%|██████████| 73/73 [14:01<00:00, 11.53s/it]


Buscando palabras...


<ipython-input-14-6c2c44de1284>:118: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3,3,3,3,4,4,4,6,7,7,7,7,7,7,7,7,7,8,8,8,8,8,9,9,9,9,9,9,9,10,10,10,10,10,10,10,10,11,11,11,11,11,11,11,11,11,12,12,12,12,12,12,12,12,12,12,12,12,12,12,13,13,13,13,13,13,13,13,14,14,14,14,14,14,14,14,14,14,14,14,14,15,15,15,15,15,16,16,17,17,17,17,17,17,17,17,17,17,17,17,17,18,18,18,18,18,19,19,19,19,19,19,19,19,19,19,19,19,19,19,20,20,20,20,20,20,20,20,20,20,20,20,21,21,21,21,21,21,21,21,21,21,21,21,21,21,22,22,22,22,22,22,22,22,22,22,23,23,23,23,23,23,23,23,23,23,23,23,24,24,24,24,25,25,25,25,25,26,26,26,26,26,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,28,28,28,28,28,28,28,28,28,29,29,29,29,29,29,29,29,29,29,30,30,30,30,30,30,30,30,30,30,31,31,31,31,31,31,31,31,31,31,31,31,31,32,32,32,32,32,32,32,32,32,32,32,32,33,33,33,33,33,33,33,33,33,33,34,34,34,34,34,34,34,34,34,34,34,35,35,35,35,35,35,35,35,35,35,35,36,36,36,36

Guardando resultados...
Archivo guardado en:  Output.xlsx
